In [1]:
import open3d as o3d
import numpy as np
import math
import copy

In [2]:
def load_point_cloud(path):
    """
    Description
    -----------
        read point cloud

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.PointCloud
    """
    cloud = o3d.io.read_point_cloud(path)
    return cloud


In [3]:
from pyquaternion import Quaternion


def matrix2rotation(matrix):
    return matrix[:3, :3]


def matrix2qua(matrix):
    qua = Quaternion(matrix=matrix2rotation(matrix))
    return qua


def matrix2trans(matrix):
    return np.array(matrix[:3, 3:].transpose())[0]


def matrix2pose(matrix):
    qua = matrix2qua(matrix)
    translation = matrix2trans(matrix)
    return np.hstack((translation, qua.elements))


def quatrans2matrix(qua, trans):
    rotaion_matrix = qua.rotation_matrix
    trans_matrix = np.array([trans]).transpose()
    tf_m = np.hstack((rotaion_matrix, trans_matrix))
    m = np.mat([[0, 0, 0, 1]])
    tf_m = np.vstack((tf_m, m))

    return tf_m


def pose2matrix(pose):
    w, x, y, z = pose[3:]
    qua = Quaternion(w=w, x=x, y=y, z=z)
    trans = pose[:3]
    return quatrans2matrix(qua, trans)


def translation2matrix(trans):
    qua = Quaternion(w=1, x=0, y=0, z=0)
    return quatrans2matrix(qua, trans)


def qua2matrix(qua):
    trans = [0, 0, 0]
    return quatrans2matrix(qua, trans)

In [4]:
def get_picked_indices(pcd):
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window(window_name="Resgistration: Pick Corresponding Points")
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    return vis.get_picked_points()


def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries(
        [source_temp, target], window_name="Registration Visualization"
    )
    return source_temp + target


# def icp(source, target, icp_threshold):
#     print('Please press Shift and click the feature points in the same order, press Q to finish')
#     picked_id_source = get_picked_indices(source)
#     print('picked_id_source: ', picked_id_source)
#     return picked_id
        
#     while True:
        
#         print('Please press Shift and click the feature points in the same order, press Q to finish')
#         picked_id_source = get_picked_indices(source)
#         print('picked_id_source: ', picked_id_source)
        
#         print('Please press Shift and click the feature points, press Q to finish')
#         picked_id_target = get_picked_indices(target)
#         print('picked_id_target: ', picked_id_target)

#         if len(picked_id_source) != len(picked_id_target):
#             print("The number of corresponding points are not the same, please re-pick")
#         else:
#             break

#     corr = np.zeros((len(picked_id_source), 2))
#     corr[:, 0] = picked_id_source
#     corr[:, 1] = picked_id_target

#     p2p = o3d.registration.TransformationEstimationPointToPoint()
#     trans_init = p2p.compute_transformation(
#         source, target, o3d.utility.Vector2iVector(corr)
#     )

#     reg_p2p = o3d.registration.registration_icp(
#         source,
#         target,
#         icp_threshold,
#         trans_init,
#         o3d.registration.TransformationEstimationPointToPoint(),
#     )

#     transform = reg_p2p.transformation
#     print("Transformation Matrix: \n{}\n".format(transform))

#     final_cloud = draw_registration_result(
#         source, target, transform)
    
#     return transform

In [5]:
def load_mesh_model(path):
    """
    Description
    -----------
        read mesh model

    Parameters
    ----------
        path: str

    Returns
    -------
        o3d.geometry.TriangleMesh
    """
    mesh = o3d.io.read_triangle_mesh(path)
    return mesh

def sample_mesh_to_cloud(mesh, sample_number=100000):
    """
    Description
    -----------
        process the cabinet mesh model, sample it as point cloud

    Parameters
    ----------
        mesh: o3d.geometry.TriangleMesh
        sample_number: int

    Returns
    -------
        o3d.geometry.PointCloud
    """
    mesh.compute_vertex_normals()
    pcd = mesh.sample_points_uniformly(number_of_points=sample_number)
    pcd.paint_uniform_color([1, 0.706, 0])  # yellow
    return pcd

In [6]:
path = "/home/bot/dev/dr_vision_lib/onsite_data/gobekli/1102/boxes_1102.stl"
mesh = load_mesh_model(path)
target = sample_mesh_to_cloud(mesh, 500000)
o3d.visualization.draw_geometries([target])

RuntimeError: [1;31m[Open3D ERROR] GLFW Error: GLX: Forward compatibility requested but GLX_ARB_create_context_profile is unavailable[0;m

In [14]:
target = load_point_cloud("/home/bot/dev/loading_projects/onsite_data/1203-2/left.pcd")
source = load_point_cloud("/home/bot/dev/loading_projects/onsite_data/1203-2/right.pcd")

In [7]:
target_ids = get_picked_indices(target)
print(len(target_ids))
print(target_ids)

11
[37399, 40772, 40836, 30490, 66834, 58007, 66719, 94475, 93487, 93021, 71956]


In [8]:
source_ids = get_picked_indices(source)
print(len(source_ids))
print(source_ids)

11
[114519, 114032, 111318, 116377, 146232, 144051, 140672, 167729, 165325, 160901, 135582]


In [15]:
corr = np.zeros((len(source_ids), 2))
corr[:, 0] = source_ids
corr[:, 1] = target_ids

p2p = o3d.registration.TransformationEstimationPointToPoint()
trans_init = p2p.compute_transformation(
    source, target, o3d.utility.Vector2iVector(corr)
)
print(trans_init)

[[ 0.99883     0.04762557  0.00839211 -0.0754092 ]
 [-0.048322    0.97610091  0.21187733 -1.68516968]
 [ 0.00189923 -0.21203496  0.97726024  0.16794097]
 [ 0.          0.          0.          1.        ]]


In [16]:
draw_registration_result(source, target, trans_init)

geometry::PointCloud with 387993 points.

In [11]:
center_point = np.array([0,0,0])
min_bound = center_point - np.asarray([2, 2, 0])
max_bound = center_point + np.asarray([2, 2, 3.5])

cropbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound, max_bound=max_bound)

source = source.crop(cropbox)
target = target.crop(cropbox)

In [17]:
reg_p2p = o3d.registration.registration_icp(
    source,
    target,
    0.005,
    trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
)

transform = reg_p2p.transformation
print("Transformation Matrix: \n{}\n".format(transform))

Transformation Matrix: 
[[ 9.98762687e-01  4.86852201e-02  1.01412232e-02 -8.30237946e-02]
 [-4.97294316e-02  9.76609307e-01  2.09191885e-01 -1.67619676e+00]
 [ 2.80540001e-04 -2.09437366e-01  9.77822024e-01  1.62564535e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



In [18]:
final_cloud = draw_registration_result(source, target, transform)

In [19]:
pose = matrix2pose(transform)
print(pose)

[-0.08302379 -1.67619676  0.16256453  0.99413204 -0.10527506  0.00247972
 -0.02474889]
